<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/38_3_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых пакетов
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libcudnn8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
tensorflow-tpu 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.17.1 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
tensorflow-tpu 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.17.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.

In [2]:
# Импорт необходимых библиотек
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
import re
import pathlib
import random
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Загрузка датасета
ds = tfds.load('web_questions', split='train', shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3778 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.38L8B0_1.0.0/web_questions-train.tfrecord*...:   …

Generating test examples...:   0%|          | 0/2032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.38L8B0_1.0.0/web_questions-test.tfrecord*...:   0…

Dataset web_questions downloaded and prepared to /root/tensorflow_datasets/web_questions/1.0.0. Subsequent calls will reuse this data.


In [4]:
# Содержимое датасета
ds

<_PrefetchDataset element_spec={'answers': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'url': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [5]:
# Выводим два примера из датасета
i = 1
for example in ds:
    if i < 3:
        print(f"Пример из обучающей выборки №{i}:")
        print(*example['answers'].numpy())
        print(example['question'].numpy())
        print(example['url'].numpy(), "\n")
        i += 1

Пример из обучающей выборки №1:
b'Joe Flacco'
b'who is the ravens quarterback 2012?'
b'http://www.freebase.com/view/en/baltimore_ravens' 

Пример из обучающей выборки №2:
b'Whig Party'
b'what political party was henry clay?'
b'http://www.freebase.com/view/en/henry_clay' 



In [6]:
# создаем списки вопросов и ответов
question_list = []
answer_list = []

i = 0
for example in ds:
    if i < 6642:
        question = example['question'].numpy()
        question_list.append(question)

        answer = example['answers'].numpy()
        answer_list.append(answer)
        i += 1

print('Размер списка вопросов: ', len(question_list))
print('Размер списка ответов: ', len(answer_list))

Размер списка вопросов:  3778
Размер списка ответов:  3778


In [7]:
# Расщепляем списки
train_examples_question = question_list[:3000]
train_examples_answer = answer_list[:3000]

val_examples_question = question_list[3001:]
val_examples_answer = answer_list[3001:]

print(f"Длина обучающей (вопросы): {len(train_examples_question)}")
print(f"Длина обучающей (ответы): {len(train_examples_answer)}")
print(f"Длина валидационной (вопросы): {len(val_examples_question)}")
print(f"Длина обучаювалидационнойщей (ответы): {len(val_examples_answer)}")

Длина обучающей (вопросы): 3000
Длина обучающей (ответы): 3000
Длина валидационной (вопросы): 777
Длина обучаювалидационнойщей (ответы): 777


In [8]:
# Выводим содержимое спика
rnd = random.randint(0, 3000)
print(train_examples_question[rnd])
print(train_examples_answer[rnd])

b'who is ellen albertini dow?'
[b'Actor']


In [9]:
# Токенизация списков вопросов и ответов
VOCAB_SIZE = 8000
bert_tokenizer_params=dict(lower_case=True)

reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = VOCAB_SIZE,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},)

In [11]:
# ???
train_answer = ds.map(lambda x: x['answers'])
train_question = ds.map(lambda x: x['question'])

print(train_answer)
print(train_question)


# в уроке 38.2 возвращает:
# <_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
# <_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [12]:
question_vocab = bert_vocab.bert_vocab_from_dataset(train_question.batch(1000).prefetch(2), **bert_vocab_args)
print(question_vocab[:10])
print(question_vocab[100:110])
print(question_vocab[1000:1010])
print(question_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', "'", '(', ')', '.', '0', '1']
['use', 'college', '##ie', 'and', 'city', 'movies', '##k', '##h', '##m', 'state']
[]
['##3', '##4', '##5', '##6', '##7', '##8', '##9', '##?', '##j', '##q']


In [14]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

write_vocab_file('question_vocab.txt', question_vocab)

In [15]:
answer_vocab = bert_vocab.bert_vocab_from_dataset(train_answer.batch(1000).prefetch(2), **bert_vocab_args)
print(answer_vocab[:10])
print(answer_vocab[100:110])
print(answer_vocab[1000:1010])
print(answer_vocab[-10:])

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [10], [batch]: [1] [Op:IteratorGetNext] name: 

In [26]:
write_vocab_file('answer_vocab.txt', answer_vocab)

In [ ]:
!ls *.txt